In [1]:
import pandas as pd
import numpy as np
import holidays
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
import matplotlib.pyplot as plt

In [2]:
BASE_PATH = r"C:\Users\fdcontreras\OneDrive - Indra\Universidad\Despliegue de Soluciones Analíticas\Proyecto_Despliegue\Proyecto_Despliegue\data\01_raw"

def load_datasets():
    files = {
        'orders': 'olist_orders_dataset.csv',
        'order_items': 'olist_order_items_dataset.csv',
        'customers': 'olist_customers_dataset.csv',
        'products': 'olist_products_dataset.csv',
        'sellers': 'olist_sellers_dataset.csv',
        'order_payments': 'olist_order_payments_dataset.csv',
        'order_reviews': 'olist_order_reviews_dataset.csv',
        'geolocation': 'olist_geolocation_dataset.csv',
        'product_translation': 'product_category_name_translation.csv'
    }
    
    datasets = {}
    for name, filename in files.items():
        filepath = os.path.join(BASE_PATH, filename)
        datasets[name] = pd.read_csv(filepath)
    
    return datasets
data = load_datasets()

for name, df in data.items():
    print(f"{name}: {df.shape}")

orders: (99441, 8)
order_items: (112650, 7)
customers: (99441, 5)
products: (32951, 9)
sellers: (3095, 4)
order_payments: (103886, 5)
order_reviews: (99224, 7)
geolocation: (1000163, 5)
product_translation: (71, 2)


In [3]:

merged_df = data['orders'].merge(
    data['order_items'], 
    on='order_id', 
    how='left'
)

print(f"Después de order_items: {merged_df.shape}")

# Agregar información de productos
merged_df = merged_df.merge(
    data['products'], 
    on='product_id', 
    how='left'
)

print(f"Después de products: {merged_df.shape}")

# Agregar traducción de categorías
merged_df = merged_df.merge(
    data['product_translation'], 
    on='product_category_name', 
    how='left'
)

print(f"Después de product_translation: {merged_df.shape}")
# Agregar información de vendedores
merged_df = merged_df.merge(
    data['sellers'], 
    on='seller_id', 
    how='left'
)

print(f"Después de sellers: {merged_df.shape}")

# Agregar información de clientes
merged_df = merged_df.merge(
    data['customers'], 
    on='customer_id', 
    how='left'
)

print(f"Después de customers: {merged_df.shape}")

# Agregar pagos
merged_df = merged_df.merge(
    data['order_payments'], 
    on='order_id', 
    how='left'
)

print(f"Después de order_payments: {merged_df.shape}")

# Agregar reviews
merged_df = merged_df.merge(
    data['order_reviews'], 
    on='order_id', 
    how='left'
)

print(f"Después de order_reviews: {merged_df.shape}")

# Agregar geolocalización de clientes
customer_geo = data['geolocation'].drop_duplicates(subset=['geolocation_zip_code_prefix'])
customer_geo = customer_geo.rename(columns={
    'geolocation_zip_code_prefix': 'customer_zip_code_prefix',
    'geolocation_lat': 'customer_lat',
    'geolocation_lng': 'customer_lng',
    'geolocation_city': 'customer_geo_city',
    'geolocation_state': 'customer_geo_state'
})

merged_df = merged_df.merge(
    customer_geo, 
    on='customer_zip_code_prefix', 
    how='left'
)

print(f"Después de customer geolocation: {merged_df.shape}")

# Agregar geolocalización de vendedores
seller_geo = data['geolocation'].drop_duplicates(subset=['geolocation_zip_code_prefix'])
seller_geo = seller_geo.rename(columns={
    'geolocation_zip_code_prefix': 'seller_zip_code_prefix',
    'geolocation_lat': 'seller_lat',
    'geolocation_lng': 'seller_lng',
    'geolocation_city': 'seller_geo_city',
    'geolocation_state': 'seller_geo_state'
})

merged_df = merged_df.merge(
    seller_geo, 
    on='seller_zip_code_prefix', 
    how='left'
)

print(f"Dataset final: {merged_df.shape}")

# Verificar información del dataset final
print("Información del dataset consolidado:")
print(f"Filas: {merged_df.shape[0]:,}")
print(f"Columnas: {merged_df.shape[1]}")
print(f"Memoria: {merged_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Después de order_items: (113425, 14)
Después de products: (113425, 22)
Después de product_translation: (113425, 23)
Después de sellers: (113425, 26)
Después de customers: (113425, 30)
Después de order_payments: (118434, 34)
Después de order_reviews: (119143, 40)
Después de customer geolocation: (119143, 44)
Dataset final: (119143, 48)
Información del dataset consolidado:
Filas: 119,143
Columnas: 48
Memoria: 224.90 MB


In [4]:
merged_df['order_purchase_timestamp'] = pd.to_datetime(merged_df['order_purchase_timestamp'])
merged_df['order_date'] = merged_df['order_purchase_timestamp'].dt.date

In [5]:
category_days = (
    merged_df
      .groupby('product_category_name_english')['order_date']
      .nunique()
      .reset_index(name='unique_days')
      .sort_values('unique_days', ascending=False)
)

pd.set_option('display.max_rows',     None)
pd.set_option('display.max_columns',  None)
pd.set_option('display.width',        200)
pd.set_option('display.max_colwidth', None)

display(category_days)

,product_category_name_english,unique_days
39,furniture_decor,607
65,sports_leisure,603
43,health_beauty,602
7,bed_bath_table,598
15,computers_accessories,593
69,toys,590
68,telephony,588
49,housewares,587
42,garden_tools,585
20,cool_stuff,584


In [6]:
MIN_UNIQUE_DAYS = 45

valid_cats = (
    category_days[
        category_days['unique_days'] >= MIN_UNIQUE_DAYS
    ]['product_category_name_english']
    .tolist()
)

print(f"Categorías válidas (≥{MIN_UNIQUE_DAYS} días): {len(valid_cats)}\n", valid_cats)

Categorías válidas (≥45 días): 58
 ['furniture_decor', 'sports_leisure', 'health_beauty', 'bed_bath_table', 'computers_accessories', 'toys', 'telephony', 'housewares', 'garden_tools', 'cool_stuff', 'auto', 'perfumery', 'watches_gifts', 'baby', 'fashion_bags_accessories', 'pet_shop', 'stationery', 'electronics', 'luggage_accessories', 'office_furniture', 'consoles_games', 'small_appliances', 'musical_instruments', 'home_appliances', 'books_general_interest', 'construction_tools_construction', 'furniture_living_room', 'home_confort', 'home_construction', 'food', 'audio', 'market_place', 'drinks', 'air_conditioning', 'fashion_shoes', 'kitchen_dining_laundry_garden_furniture', 'home_appliances_2', 'fixed_telephony', 'books_technical', 'food_drink', 'industry_commerce_and_business', 'art', 'costruction_tools_garden', 'agro_industry_and_commerce', 'construction_tools_lights', 'construction_tools_safety', 'signaling_and_security', 'fashion_underwear_beach', 'christmas_supplies', 'computers', 

In [7]:
daily = (
    merged_df
    .groupby(['product_category_name_english','order_date'])
    .size()
    .reset_index(name='units_sold')
)


daily.head()


,product_category_name_english,order_date,units_sold
0,agro_industry_and_commerce,2017-01-23,2
1,agro_industry_and_commerce,2017-01-31,1
2,agro_industry_and_commerce,2017-02-05,1
3,agro_industry_and_commerce,2017-02-08,1
4,agro_industry_and_commerce,2017-02-12,1


In [8]:
# Filtrar categorías con al menos 45 días de ventas
daily_filtered = daily[
    daily['product_category_name_english'].isin(valid_cats)
].copy()

print(f"daily_filtered shape: {daily_filtered.shape}")
daily_filtered['product_category_name_english'].nunique()

daily_filtered shape: (18180, 3)


58

In [9]:
pd.set_option('display.max_rows',   None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',      200)

display(daily_filtered.head(20))

,product_category_name_english,order_date,units_sold
0,agro_industry_and_commerce,2017-01-23,2
1,agro_industry_and_commerce,2017-01-31,1
2,agro_industry_and_commerce,2017-02-05,1
3,agro_industry_and_commerce,2017-02-08,1
4,agro_industry_and_commerce,2017-02-12,1
5,agro_industry_and_commerce,2017-02-13,1
6,agro_industry_and_commerce,2017-02-16,1
7,agro_industry_and_commerce,2017-02-18,1
8,agro_industry_and_commerce,2017-02-21,3
9,agro_industry_and_commerce,2017-03-17,1


In [10]:
TRAIN_DAYS     = 30
TEST_DAYS      = 15
TS_SPLITS      = 5
RANDOM_STATE   = 42

years  = sorted(daily['order_date'].apply(lambda d: d.year).unique())
br_hols= holidays.Brazil(years=years)
results_xgb = []
tscv = TimeSeriesSplit(n_splits=TS_SPLITS)

In [11]:
# Loop para cada categoría válida
for cat in valid_cats:
    df_cat = daily[daily['product_category_name_english']==cat].copy()
    df_cat['order_date'] = pd.to_datetime(df_cat['order_date'])
    df_cat.set_index('order_date', inplace=True)

    # Feature engineering diario
    df_cat['lag1']  = df_cat['units_sold'].shift(1)
    df_cat['roll7'] = df_cat['units_sold'].rolling(7).mean()
    idx = df_cat.index
    df_cat['dow_sin'] = np.sin(2*np.pi * idx.weekday / 7)
    df_cat['dow_cos'] = np.cos(2*np.pi * idx.weekday / 7)
    df_cat['mes_sin'] = np.sin(2*np.pi * idx.month   / 12)
    df_cat['mes_cos'] = np.cos(2*np.pi * idx.month   / 12)
    df_cat['festivo']= idx.to_series().isin(br_hols).astype(int)
    # Black Friday
    bf_dates=[]
    for y in idx.year.unique():
        nov = pd.date_range(f"{y}-11-01", f"{y}-11-30", freq="D")
        th  = nov[nov.weekday==3]
        bf_dates.append((th[3]+pd.Timedelta(days=1)).date())
    df_cat['bf'] = idx.to_series().apply(lambda d: d.date() in bf_dates).astype(int)

    df_feat = df_cat.dropna().drop(columns=['product_category_name_english'])
    if len(df_feat)<=TRAIN_DAYS:
        continue

    X = df_feat.drop(columns=['units_sold'])
    y = df_feat['units_sold'].values
    X_train, X_test = X.iloc[:TRAIN_DAYS], X.iloc[TRAIN_DAYS:TRAIN_DAYS+TEST_DAYS]
    y_train, y_test = y[:TRAIN_DAYS], y[TRAIN_DAYS:TRAIN_DAYS+TEST_DAYS]
    if y_test.sum()==0:
        continue

    # Espacio de búsqueda para XGBoost
    param_dist = {
        'n_estimators':    [100, 200, 500],
        'max_depth':       [3, 5, 7, 10],
        'learning_rate':   [0.01, 0.05, 0.1],
        'subsample':       [0.6, 0.8, 1.0],
        'colsample_bytree':[0.6, 0.8, 1.0],
        'reg_alpha':       [0, 0.1, 1],
        'reg_lambda':      [1, 5, 10]
    }

    search = RandomizedSearchCV(
        XGBRegressor(
            objective='reg:squarederror',
            random_state=RANDOM_STATE,
            n_jobs=-1
        ),
        param_distributions=param_dist,
        n_iter=30,
        cv=tscv,
        scoring='neg_mean_squared_error',
        verbose=0,
        random_state=RANDOM_STATE
    )
    search.fit(X_train, y_train)
    best = search.best_estimator_

    # Predicción y métricas
    preds = best.predict(X_test)
    mae   = mean_absolute_error(y_test, preds)
    rmse  = np.sqrt(mean_squared_error(y_test, preds))
    mape  = np.mean(np.abs((y_test - preds)/y_test)[y_test>0])*100

    results_xgb.append({
        'category':       cat,
        'MAE':            mae,
        'RMSE':           rmse,
        'MAPE (%)':       mape,
        **search.best_params_
    })


res_xgb = pd.DataFrame(results_xgb)\
             .sort_values('MAPE (%)')\
             .reset_index(drop=True)
pd.set_option('display.max_rows', None)
display(res_xgb)

,category,MAE,RMSE,MAPE (%),subsample,reg_lambda,reg_alpha,n_estimators,max_depth,learning_rate,colsample_bytree
0,furniture_bedroom,0.156114,0.330914,9.645450,1.0,10,0.1,100,10,0.05,0.8
1,books_imported,0.270288,0.509371,19.096820,1.0,5,0.0,200,5,0.05,1.0
2,books_technical,0.266377,0.515713,19.971657,1.0,1,0.0,500,7,0.10,1.0
3,dvds_blu_ray,0.578407,1.360224,20.216783,0.8,1,1.0,200,7,0.10,0.6
4,tablets_printing_image,0.341177,0.558615,20.763008,0.6,10,0.0,100,3,0.01,1.0
5,home_appliances_2,0.548413,0.941965,26.664250,0.6,10,0.0,100,3,0.01,1.0
6,kitchen_dining_laundry_garden_furniture,0.414929,0.571251,27.397960,0.8,10,1.0,500,10,0.01,0.6
7,food,0.702052,1.305866,33.145092,1.0,1,0.0,500,7,0.10,1.0
8,cine_photo,0.561768,0.760202,34.064875,0.6,10,1.0,200,10,0.10,0.6
9,fashion_underwear_beach,0.759562,1.140116,34.912151,0.6,10,0.0,100,3,0.01,1.0
